In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from sklearn.metrics import mean_absolute_error

df_user_item_rating_train = pd.read_csv(".\\ml-100k_u1\\u1.base",delimiter='\t',names=['user_id','item_id','rating','timestamp'])
df_item_rating_train = df_user_item_rating_train.groupby('item_id',as_index=False)['rating'].mean()

df_user_item_rating_test = pd.read_csv(".\\ml-100k_u1\\u1.test",delimiter='\t',names=['user_id','item_id','rating','timestamp'])
df_item_rating_test = df_user_item_rating_test.groupby('item_id',as_index=False)['rating'].mean()

df_items = pd.read_csv(".\\ml-100k_u1\\u.item",delimiter='|',names=['item_id','movie title','release date','video release date','IMDb URL'
          ,'unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western'])



merged_item_info_rating_train = pd.merge(df_item_rating_train,df_items)
merged_item_info_rating_test = pd.merge(df_item_rating_test,df_items)

df_train_x_train = merged_item_info_rating_train[['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']]

df_train_y_train = merged_item_info_rating_train[['item_id','rating']].set_index(['item_id'])

df_train_x_test = merged_item_info_rating_test[['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']]

df_train_y_test = merged_item_info_rating_test[['item_id','rating']].set_index(['item_id'])


# model = LinearRegression()
# model.fit(df_train_x_train, df_train_y_train)
# model.score(df_train_x_train, df_train_y_train)

# y_pred = model.predict(df_train_x_test)

df_train_y_test['prediction'] = df_train_y_train['rating']
df_train_y_test = df_train_y_test.dropna()

MAE = mean_absolute_error(df_train_y_test['rating'], df_train_y_test['prediction'])
display(MAE)



0.47400905317090192

In [18]:
sorted_df_train_y_test = df_train_y_test.sort_values(by = 'prediction',ascending=False)



# df_train_y_test.index
lst = {}
lst_test_actual = {}
predicted_recommand = {}
random_recommand = {} 

user_precision_predicted = []
user_recall_predicted = []
user_precision_random = []
user_recall_random = []

user_data_frame = []


for user in df_user_item_rating_test['user_id'].unique():
    lst[user] = df_user_item_rating_train.loc[df_user_item_rating_train['user_id'] == user]['item_id']
    predicted_recommand[user] = sorted_df_train_y_test.loc[~sorted_df_train_y_test.index.isin(lst[user])].head(20).index
    random_recommand[user] = sorted_df_train_y_test.loc[~sorted_df_train_y_test.index.isin(lst[user])].sample(20).index
    lst_test_actual[user] = df_user_item_rating_test.loc[df_user_item_rating_test['user_id'] == user]['item_id']
    user_precision_predicted.append(len(set(lst_test_actual[user]).intersection(predicted_recommand[user])) / 20.0)
    user_recall_predicted.append(len(set(lst_test_actual[user]).intersection(predicted_recommand[user])) / (len(lst_test_actual[user]) * 1.0))
    user_precision_random.append(len(set(lst_test_actual[user]).intersection(random_recommand[user])) / 20.0)
    user_recall_random.append(len(set(lst_test_actual[user]).intersection(random_recommand[user])) / (len(lst_test_actual[user]) * 1.0))
    user_data_frame.append(user)

raw_data = {}
raw_data['user_id'] = user_data_frame
raw_data['precision_predicted'] = user_precision_predicted
raw_data['recall_predicted'] = user_recall_predicted
raw_data['precision_random'] = user_precision_random
raw_data['recall_random'] = user_recall_random

df_analitics = pd.DataFrame(data = raw_data,index = [user_data_frame],columns = ['user_id','precision_predicted','recall_predicted','precision_random','recall_random'])
df_analitics = df_analitics.set_index(['user_id'])
display(df_analitics)



,precision_predicted,recall_predicted,precision_random,recall_random
user_id,,,,
1,0.25,0.036496,0.10,0.014599
2,0.05,0.045455,0.00,0.000000
3,0.05,0.038462,0.00,0.000000
4,0.10,0.200000,0.00,0.000000
5,0.05,0.011905,0.15,0.035714
6,0.40,0.079208,0.15,0.029703
7,0.25,0.026316,0.15,0.015789
8,0.05,0.034483,0.00,0.000000
9,0.00,0.000000,0.00,0.000000


In [31]:
df_users = pd.read_csv(".\\ml-100k_u1\\u.user",delimiter='|',names=['user_id','age','gender','occupation','zip_code'])

#train 
df_users_base=pd.merge(df_users,df_user_item_rating_train)

df_male_rating=df_users_base.loc[df_users_base['gender']== 'M'][['item_id','rating']]
df_female_rating=df_users_base.loc[df_users_base['gender']== 'F'][['item_id','rating']]

df_male_mean_train=df_male_rating.groupby('item_id',as_index=False).mean()
df_female_mean_train=df_female_rating.groupby('item_id',as_index=False).mean()

#test
df_users_test=pd.merge(df_users,df_user_item_rating_test)

df_male_rating=df_users_test.loc[df_users_base['gender']== 'M'][['item_id','rating']]
df_female_rating=df_users_test.loc[df_users_base['gender']== 'F'][['item_id','rating']]

df_male_mean_test=df_male_rating.groupby('item_id',as_index=False).mean().set_index(['item_id'])
df_female_mean_test=df_female_rating.groupby('item_id',as_index=False).mean().set_index(['item_id'])


#compare male
df_male_mean_test['prediction'] = df_male_mean_train['rating']
df_male_mean_test_mae = df_male_mean_test.dropna()

MAE = mean_absolute_error(df_male_mean_test_mae['rating'], df_male_mean_test_mae['prediction'])
display(MAE)

#compare female
df_female_mean_test['prediction'] = df_female_mean_train['rating']
df_female_mean_test_mae = df_female_mean_test.dropna()

MAE = mean_absolute_error(df_female_mean_test_mae['rating'], df_female_mean_test_mae['prediction'])
display(MAE)


0.78622490582146476

0.83390008753034328

In [30]:
# sorted_df_train_y_test = df_female_mean_test.sort_values(by = 'prediction',ascending=False)

sorted_male_female=[df_male_mean_test.sort_values(by = 'prediction',ascending=False),
                    df_female_mean_test.sort_values(by = 'prediction',ascending=False)]
# print df_male_mean_test.shape
# print df_female_mean_test.shape
# df_train_y_test.index
lst = {}
lst_test_actual = {}
predicted_recommand = {}
random_recommand = {} 

user_precision_predicted = []
user_recall_predicted = []
user_precision_random = []
user_recall_random = []

user_data_frame = []
male_female=['M','F']
i=0

for group in sorted_male_female:
    for user in df_users.loc[df_users['gender']==male_female[i]]['user_id'].unique():
        lst[user] = df_user_item_rating_train.loc[df_user_item_rating_train['user_id'] == user]['item_id']
        predicted_recommand[user] = group.loc[~group.index.isin(lst[user])].head(20).index
        random_recommand[user] = group.loc[~group.index.isin(lst[user])].sample(20).index
        lst_test_actual[user] = df_user_item_rating_test.loc[df_user_item_rating_test['user_id'] == user]['item_id']
        if len(lst_test_actual[user])>0:
            user_precision_predicted.append(len(set(lst_test_actual[user]).intersection(predicted_recommand[user])) / 20.0)
            user_recall_predicted.append(len(set(lst_test_actual[user]).intersection(predicted_recommand[user])) / (len(lst_test_actual[user]) * 1.0))
            user_precision_random.append(len(set(lst_test_actual[user]).intersection(random_recommand[user])) / 20.0)
            user_recall_random.append(len(set(lst_test_actual[user]).intersection(random_recommand[user])) / (len(lst_test_actual[user]) * 1.0))
            user_data_frame.append(user)

    raw_data = {}
    raw_data['user_id'] = user_data_frame
    raw_data['precision_predicted'] = user_precision_predicted
    raw_data['recall_predicted'] = user_recall_predicted
    raw_data['precision_random'] = user_precision_random
    raw_data['recall_random'] = user_recall_random

    df_analitics = pd.DataFrame(data = raw_data,index = [user_data_frame],columns = ['user_id','precision_predicted','recall_predicted','precision_random','recall_random'])
    df_analitics = df_analitics.set_index(['user_id'])
    display(df_analitics)
    i+=1



,precision_predicted,recall_predicted,precision_random,recall_random
user_id,,,,
1,0.00,0.000000,0.05,0.007299
3,0.00,0.000000,0.00,0.000000
4,0.00,0.000000,0.05,0.100000
6,0.00,0.000000,0.00,0.000000
7,0.10,0.010526,0.30,0.031579
8,0.00,0.000000,0.05,0.034483
9,0.00,0.000000,0.00,0.000000
10,0.05,0.011111,0.10,0.022222
13,0.25,0.019011,0.25,0.019011


,precision_predicted,recall_predicted,precision_random,recall_random
user_id,,,,
1,0.00,0.000000,0.05,0.007299
3,0.00,0.000000,0.00,0.000000
4,0.00,0.000000,0.05,0.100000
6,0.00,0.000000,0.00,0.000000
7,0.10,0.010526,0.30,0.031579
8,0.00,0.000000,0.05,0.034483
9,0.00,0.000000,0.00,0.000000
10,0.05,0.011111,0.10,0.022222
13,0.25,0.019011,0.25,0.019011
